In [4]:
# =======================
# RSA PKCS#1 v1.5 Sign/Verify
# =======================
import hashlib
import secrets
from sympy import nextprime
import math

# --- Helpers ---
def int_to_bytes(i, length):
    return i.to_bytes(length, 'big')

def bytes_to_int(b):
    return int.from_bytes(b, 'big')

MD5_PREFIX = bytes.fromhex('3020300c06082a864886f70d020505000410')

def emsa_pkcs1_v1_5_encode(digest_bytes, k):
    t = MD5_PREFIX + digest_bytes
    ps_len = k - len(t) - 3
    if ps_len < 8:
        raise ValueError("Modulus too small for PKCS#1 v1.5")
    return b'\x00\x01' + b'\xff'*ps_len + b'\x00' + t

def egcd(a,b):
    if b==0: return a,1,0
    g,x1,y1 = egcd(b, a%b)
    return g, y1, x1 - (a//b)*y1

def modinv(a,m):
    g,x,y = egcd(a,m)
    if g != 1: raise ValueError("No inverse")
    return x % m

def generate_prime_fast(bits):
    start = secrets.randbits(bits) | (1 << (bits-1)) | 1
    return int(nextprime(start))

def generate_rsa_key(bits=512):
    half = bits // 2
    p = generate_prime_fast(half)
    q = generate_prime_fast(half)
    while q == p:
        q = generate_prime_fast(half)
    n = p * q
    phi = (p-1)*(q-1)
    e = 65537
    if math.gcd(e, phi) != 1:
        e = 3
        while math.gcd(e, phi) != 1:
            e += 2
    d = modinv(e, phi)
    return {'n':n,'e':e,'d':d}

def sign(message_bytes, d, n):
    k = (n.bit_length() + 7)//8
    digest = hashlib.md5(message_bytes).digest()
    em = emsa_pkcs1_v1_5_encode(digest, k)
    s_int = pow(bytes_to_int(em), d, n)
    return int_to_bytes(s_int, k), digest

def verify(message_bytes, signature_bytes, e, n):
    k = (n.bit_length() + 7)//8
    if len(signature_bytes) != k:
        return False
    s_int = bytes_to_int(signature_bytes)
    m_int = pow(s_int, e, n)
    em = int_to_bytes(m_int, k)
    if not (em[0] == 0x00 and em[1] == 0x01):
        return False
    try:
        sep = em.index(0x00, 2)
    except ValueError:
        return False
    ps = em[2:sep]
    if len(ps) < 8 or any(x != 0xff for x in ps):
        return False
    digest_info = em[sep+1:]
    if not digest_info.startswith(MD5_PREFIX):
        return False
    recovered = digest_info[len(MD5_PREFIX):]
    actual = hashlib.md5(message_bytes).digest()
    return recovered == actual

# -------- RUN --------
# Randomly choose bits: 512, 768, 1024
bits = secrets.choice([512, 768, 1024])
message = input("Enter message to sign: ").encode('utf-8')

key = generate_rsa_key(bits)
signature, digest = sign(message, key['d'], key['n'])

sender_data = {'message': message, 'signature': signature, 'n': key['n'], 'e': key['e']}

print(f"\nRandom RSA modulus selected: {bits} bits")
print("Message:", message.decode())
print("MD5 Digest:", digest.hex())
print("Signature (hex, first 120 chars):", signature.hex()[:120]+"...")

# -------- Verification --------
valid = verify(sender_data['message'], sender_data['signature'], sender_data['e'], sender_data['n'])
tampered = sender_data['message'] + b"!"
valid2 = verify(tampered, sender_data['signature'], sender_data['e'], sender_data['n'])

print("\nMessage received:", sender_data['message'].decode())
print("Signature verification:", "VALID ✅" if valid else "INVALID ❌")
print("Tampered message verification:", "VALID ✅" if valid2 else "INVALID ❌")



Random RSA modulus selected: 1024 bits
Message: Hello World
MD5 Digest: b10a8db164e0754105b7a99be72e3fe5
Signature (hex, first 120 chars): 3406d75ebaaacf5d1a37693ebbd558e7b3c8c1b3f8ba71823afc67279b0d2fff54274b5635e0670e6257465ac95f512694cd679cbfb580c7611e196e...

Message received: Hello World
Signature verification: VALID ✅
Tampered message verification: INVALID ❌
